In [24]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [25]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max_length=1000)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(text_splitter, OllamaEmbeddings(model="llama3.2:1b"))
retriever = vectordb.as_retriever()

In [27]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(
    retriever,
    name="langsmith_search",
    description="Search for information about LangSmith. For any questions about LangSmith you must use this tool."
)


In [28]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max_length=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [29]:
tools = [tool, retrieval_tool, arxiv]